In [1]:
import os
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
import math
from itertools import product
import argparse
import sys
from utils_cnn_CS import *
import calendar
import time

C:\Users\vipuser\.conda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\vipuser\.conda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\vipuser\.conda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\vipuser\.conda\envs\tensorflow-gpu\lib\s

In [2]:
def process_eachseq(seq,pssmfile,mask_seq,new_pssms):
    if os.path.exists(pssmfile):  #如果pssm文件存在
        print("found " + pssmfile + "\n")  #输出找到pssm文件+换行
        pssm = readPSSM(pssmfile)  #读取pssm文件
    else:  #否则
        print("using Blosum62\n")  #输出使用Blosum62+换行
        #pssm = convertSampleToBlosum62(seq)  #把Blosum62矩阵当作pssm用
        pssm = convertSampleToCBOW(seq)
    pssm = pssm.astype(float)  #对pssm的数据类型转换为浮点型
    PhyChem = convertSampleToPhysicsVector_pca(seq)  #将样本转化为物理向量
    pssm = np.concatenate((PhyChem, pssm), axis=1)  #物化指标和pssm对应行进行数组拼接
    seql = len(seq)   #序列长度  
    if seql <= 1000:  #如果序列长度小于等于1000
        padnum = 1000 - seql  #pad大小为1000-序列长度
        padmatrix = np.zeros([padnum, 25])  #pad矩阵为行数为padnum，列数为25的全0矩阵，即用0填充不足的地方
        pssm = np.concatenate((pssm, padmatrix), axis=0)  #物化指标和pssm进行数组拼接 
        new_pssms.append(pssm)  #新的pssm空列表中添加pssm矩阵
        mask_seq.append(gen_mask_mat(seql, padnum))  #mask序列空列表添加gen_mask矩阵，序列长度为行数，padnum为列数？？？
    else:  #如果序列长度大于1000
        pssm = np.concatenate((pssm[0:500, :], pssm[seql - 500:seql, :]), axis=0)  #pssm矩阵为前500行和后500行矩阵的拼接
        new_pssms.append(pssm)  #新的pssm空列表中添加pssm矩阵
        mask_seq.append(gen_mask_mat(1000, 0))  #mask序列空列表添加1000行0列的？？？gen_mask矩阵

In [3]:
def endpad(seqfile, labelfile, pssmdir="", npzfile = ""): #定义endpad(序列文件，标签文件，pssm路径，npz文件)
    if not os.path.exists(npzfile):  #如果npz文件不存在，建立新的pssm空列表，标签空列表，mask序列空列表，id空列表
        new_pssms = []
        labels = []
        mask_seq = []
        ids=[]
        seqs=[]
        f = open(seqfile, "r")  #f为打开序列文件
        f2 = open(labelfile, "r")  #f2为打开标签文件
        line = f.readline()  #读取序列文件的第一行
        while line != '':
            pssmfile = pssmdir + line[1:].strip() + "_pssm.txt"  #pssm文件名=pssm地址+id名+_pssm.txt
            if line[0] == '>':  #如果该行第一个字符为>
                id = line.strip()[1:]  #id为去掉>的字符
                ids.append(id)   #在id空列表中添加id
            label = f2.readline().strip()  #标签为f2（标签文件）中去掉首尾空格的内容
            labels.append(label)  #在标签空列表中添加标签
            seq = f.readline().strip()  #第一次seq为第2行的内容，实际seq为>行的下一行
            #seql = len(seq)   #序列长度  
            process_eachseq(seq,pssmfile,mask_seq,new_pssms)
            line = f.readline()  #继续读取下一行，即>行
        x = np.array(new_pssms)  #把new_pssms列表变为数组，赋给x
        y = [convertlabels_to_categorical(i) for i in labels]  #把标签列表转化为类别(i)
        y = np.array(y)  #再把类别转化为数组
        mask = np.array(mask_seq)  #把mask_seq（标注的序列？）转化为数组
        np.savez(npzfile, x=x, y=y, mask=mask, ids=ids)  #保存多个数组到同一个文件中,保存格式是.npz
        return [x, y, mask,ids]  #返回pssm矩阵，类别，标注序列，名字id
    else:  #如果上述都存在，直接转化为数组
        mask = np.load(npzfile)['mask']
        x = np.load(npzfile)['x']
        y = np.load(npzfile)['y']
        ids=np.load(npzfile)['ids']
        return [x, y, mask,ids]

In [4]:
def train_MULocDeep(lv1_dir,lv2_dir,pssm_dir,output_dir,foldnum):
    # get small data
    [train_x, train_y, train_mask, train_ids] = endpad(
        lv2_dir+"lv2_train_fold" + str(foldnum) + "_seq",
        lv2_dir+"lv2_train_fold" + str(foldnum) + "_lab",
        pssm_dir,
        "D:/mulocdeep/mul_data/lv2_train_fold"+str(foldnum)+"_seq.npz")
    [val_x, val_y, val_mask,val_ids] = endpad(
        lv2_dir+"lv2_val_fold" + str(foldnum) + "_seq",
        lv2_dir+"lv2_val_fold" + str(foldnum) + "_lab",
        pssm_dir,
        "D:/mulocdeep/mul_data/lv2_val_fold"+str(foldnum)+"_seq.npz")

    # get big data 训练10分类的多分类
    [train_x_big, train_y_big, train_mask_big, train_ids_big] = endpad(
        lv1_dir + "lv1_train_fold" + str(foldnum) + "_seq",
        lv1_dir + "lv1_train_fold" + str(foldnum) + "_lab",
        pssm_dir,
        "D:/mulocdeep/mul_data/lv1_train_fold" + str(foldnum) + "_seq.npz")

    [val_x_big, val_y_big, val_mask_big, val_ids_big] = endpad(
        lv1_dir + "lv1_val_fold" + str(foldnum) + "_seq",
        lv1_dir + "lv1_val_fold" + str(foldnum) + "_lab",
        pssm_dir,
        "D:/mulocdeep/mul_data/lv1_val_fold" + str(foldnum) + "_seq.npz")

    batch_size = 128
    print("doing " + str(foldnum) + "th fold")
    model_big, model_small = singlemodel(train_x)  #模型为singlemodel

    filepath_acc_big_lv1 = output_dir+"fold" + str(
        foldnum) + "_big_lv1_acc-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

    filepath_acc_small_lv2 = output_dir+"fold" + str(
        foldnum) + "_small_lv2_acc-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

    filepath_loss_big_lv1 = output_dir+"fold" + str(
        foldnum) + "_big_lv1_loss-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

    filepath_loss_small_lv2 = output_dir+"fold" + str(
        foldnum) + "_small_lv2_loss-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"


    checkpoint_acc_big_lev1 = ModelCheckpoint(filepath_acc_big_lv1, monitor='val_accuracy', save_best_only=True,
                                          mode='max',
                                          save_weights_only=True, verbose=1)

    checkpoint_acc_small_lev2 = ModelCheckpoint(filepath_acc_small_lv2, monitor='val_lev2_accuracy', save_best_only=True,
                                          mode='max',
                                          save_weights_only=True, verbose=1)
    
    checkpoint_loss_big_lev1 = ModelCheckpoint(filepath_loss_big_lv1, monitor='val_loss', save_best_only=True,
                                          mode='min',
                                          save_weights_only=True, verbose=1)
    
    checkpoint_loss_small_lev2 = ModelCheckpoint(filepath_loss_small_lv2, monitor='val_lev2_loss', save_best_only=True,
                                          mode='min',
                                          save_weights_only=True, verbose=1)
    
    
    for i in range(80):
        # train small model
        print("epoch "+str(i)+"\n")
        '''fitHistory_batch_small = model_small.fit([train_x, train_mask.reshape(-1, 1000, 1)],
                                                 [train_y,getTrue4out1(train_y)],
                                                 batch_size=batch_size, epochs=1,
                                                 validation_data=(
                                                 [val_x, val_mask.reshape(-1, 1000, 1)], [val_y,getTrue4out1(val_y)]),
                                                 callbacks=[checkpoint_acc_small_lev2,checkpoint_loss_small_lev2],verbose=1)'''
        
        # train big model  
        fitHistory_batch_big = model_big.fit([train_x_big, train_mask_big.reshape(-1, 1000, 1)],
                                             [getTrue4out1(train_y_big)],  #为何大模型没有train_y_big
                                             batch_size=batch_size, epochs=1,  #等于1？？
                                             validation_data=(
                                             [val_x_big, val_mask_big.reshape(-1, 1000, 1)], [getTrue4out1(val_y_big)]),  #也没有val_y_big
                                             callbacks=[checkpoint_acc_big_lev1,checkpoint_loss_big_lev1], verbose=1)


In [5]:
def train_var(input_var,pssm_dir,output_dir,foldnum):
    # get small data
    [train_x,train_y,train_mask,train_ids]=endpad(input_var+"deeploc_40nr_train_fold"+str(foldnum)+"_seq",
                                        input_var+"deeploc_40nr_train_fold"+str(foldnum)+"_label",
                                        pssm_dir,
                                        "D:/deeploc/deeploc_40nr_8folds/train_fold"+str(foldnum)+"_seq.npz")
    [val_x,val_y,val_mask,val_ids]=endpad(input_var+"deeploc_40nr_val_fold"+str(foldnum)+"_seq",
                                  input_var+"deeploc_40nr_val_fold"+str(foldnum)+"_label",
                                  pssm_dir,
                                  "D:/deeploc/deeploc_40nr_8folds/val_fold"+str(foldnum)+"_seq.npz")
    batch_size = 128
    print("doing " + str(foldnum) + "th fold")
    model = var_model(train_x)   #这里的模型是var_model

    filepath_acc = output_dir+"fold" + str(foldnum) + "acc-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
    checkpoint_acc = ModelCheckpoint(filepath_acc, monitor='val_accuracy', save_best_only=True, mode='max',
                                 save_weights_only=True, verbose=1)
    fitHistory_batch = model.fit([train_x,train_mask.reshape(-1,1000,1)],getTrue4out1(train_y),
                                 batch_size=batch_size, epochs=20,
                                 validation_data=([val_x,val_mask.reshape(-1,1000,1)], getTrue4out1(val_y)),
                                 callbacks=[checkpoint_acc],verbose=1)

In [6]:
 '''我们常常可以把argparse的使用简化成下面四个步骤
       1：import argparse
       2：parser = argparse.ArgumentParser()
       3：parser.add_argument()
       4：parser.parse_args()
       上面四个步骤解释如下：首先导入该模块；然后创建一个解析对象；然后向该对象中添加你要关注的命令行参数和选项，
       每一个add_argument方法对应一个你要关注的参数或选项；最后调用parse_args()方法进行解析；解析成功之后即可使用'''
    
def main():
    #加default
    # description= 这个参数简要描述这个程度做什么以及怎么做
    parser=argparse.ArgumentParser(
        description='MULocDeep: interpretable protein localization classifier at sub-cellular and sub-organellar levels')
    #MULocDeep_model  UniLoc-train-20nr
    #--lv1_input_dir/--lv2_input_dir 亚细胞训练数据，包含8折蛋白质序列和标记  需自己添加
    parser.add_argument('--lv1_input_dir', dest='lv1_dir', type=str, 
                        default="D:/mulocdeep/mul_data",
                        help='sub-cellular training data, contains 8 folds protein sequences and labels', required=False)
    parser.add_argument('--lv2_input_dir', dest='lv2_dir', type=str,
                       default="D:/mulocdeep/mul_data",
                       help='sub-cellular training data, contains 8 folds protein sequences and labels', required=False)
    #--MULocDeep_model 添加它来训练MULocDeep模型，否则训练一个var模型
    parser.add_argument('--MULocDeep_model', dest='modeltype', action='store_false',  #触发，store_true会触发DeepLoc
                        #如果是store_false,则默认值是True，如果是store_true,则默认值是False  
                        help='Add this to train the MULocDeep model, otherwise train a variant model', required=False)
    #--model_output 受过训练的模型存储的目录的名称  需自己添加
    parser.add_argument('--model_output', dest='outputdir', type=str, 
                       default="D:/mulocdeep/lv1_result11",
                       help='the name of the directory where the trained model stores', required=False)  #由True改成False
    
    parser.add_argument('-existPSSM', dest='existPSSM', type=str,
                        default="D:/mulocdeep/mulocdeep_pssm_empty",
                        help='the name of the existing PSSM directory if there is one.', required=False)
    
    #var_model  deeploc_40nr_8folds
    #--input_dir 训练var模型的数据，包含8折蛋白质序列和标记  需自己添加
    parser.add_argument('--input_var', dest='var_dir', type=str,
                        default="D:/deeploc/deeploc_40nr_8folds",
                        help='data for traing the variant model, contains 8 folds protein sequences and labels', required=False)
    #改true  并且还需要加一个model_ouput  一个是deeploc  一个是MULocDeep
    parser.add_argument('--var_model_output', dest='var_outputdir', type=str, help='the name of the directory where the trained model stores', 
                        default="D:/deeploc/var_model_result1",
                        required=False)  #由True改成False
    parser.add_argument('-var_existPSSM', dest='var_existPSSM', type=str,
                        default="D:/deeploc/deeploc_pssm",
                        help='the name of the existing PSSM directory if there is one.', required=False)
    parser.set_defaults(feature=True)
    #args = parser.parse_args()   #改
    args = parser.parse_known_args()[0]   #jupyter下运行解析需要此代码
    model_type=args.modeltype
    input_lv1=args.lv1_dir
    input_lv2 = args.lv2_dir
    outputdir=args.outputdir
    existPSSM = args.existPSSM
    input_var=args.var_dir
    var_outputdir=args.var_outputdir
    var_existPSSM = args.var_existPSSM

    if model_type==True:
        if not input_lv1[len(input_lv1) - 1] == "/":
            input_lv1 = input_lv1 + "/"
        if not input_lv2[len(input_lv2) - 1] == "/":
            input_lv2 = input_lv2 + "/"
        if not outputdir[len(outputdir) - 1] == "/":
            outputdir = outputdir + "/"
        if not os.path.exists(outputdir):
            os.mkdir(outputdir)
        if existPSSM != "":
            if not existPSSM[len(existPSSM) - 1] == "/":
                existPSSM = existPSSM + "/"
        if ((existPSSM == "") or (not os.path.exists(existPSSM))):
            ts = calendar.timegm(time.gmtime())
            pssmdir = outputdir + str(ts) + "_pssm/"
            if not os.path.exists(pssmdir):
                os.makedirs(pssmdir)
            process_input_train(input_lv1 + "lv1_train.txt", pssmdir)
            process_input_train(input_lv2 + "lv2_train.txt", pssmdir)
            for foldnum in range(8):
                train_MULocDeep(input_lv1, input_lv2, pssmdir, outputdir, foldnum)
        else:
            for foldnum in range(8):
                train_MULocDeep(input_lv1, input_lv2, existPSSM, outputdir, foldnum)
    elif model_type==False:
        if not input_var[len(input_var) - 1] == "/":
            input_var = input_var + "/"
        if not var_outputdir[len(var_outputdir) - 1] == "/":
            var_outputdir = var_outputdir + "/"
        if not os.path.exists(var_outputdir):
            os.mkdir(var_outputdir)
        if existPSSM != "":
            if not var_existPSSM[len(var_existPSSM) - 1] == "/":
                var_existPSSM = var_existPSSM + "/"
        if ((var_existPSSM == "") or (not os.path.exists(var_existPSSM))):
            ts = calendar.timegm(time.gmtime())
            pssmdir = var_outputdir + str(ts) + "_pssm/"
            if not os.path.exists(pssmdir):
                os.makedirs(pssmdir)
            process_input_train(input_var + "processed_deeploc_train_S_seq", pssmdir)
            for foldnum in range(8):
                train_var(input_var, pssmdir, var_outputdir, foldnum)
        else:
            for foldnum in range(8):
                train_var(input_var, var_existPSSM, var_outputdir, foldnum)

In [7]:
if __name__ == "__main__":
    main()

doing 0th fold
Instructions for updating:
Colocations handled automatically by placer.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 1000, 25)     0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1000, 25)     0           Input[0][0]                      
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1000, 1)      0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1000, 25)     0           dropout_1[0][0]                  
     

Instructions for updating:
Use tf.cast instead.
Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 131s 8ms/step - loss: 0.7595 - accuracy: 0.6682 - val_loss: 0.6578 - val_accuracy: 0.7886

Epoch 00001: val_accuracy improved from -inf to 0.78863, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 0.65775, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_loss-weights.hdf5
epoch 1

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 133s 8ms/step - loss: 0.6389 - accuracy: 0.8159 - val_loss: 0.6341 - val_accuracy: 0.8317

Epoch 00001: val_accuracy improved from 0.78863 to 0.83174, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.65775 to 0.63405, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_loss-weights.hdf5
epoch 2

Train on 17138 samp

17138/17138 [==============================] - 125s 7ms/step - loss: 0.3251 - accuracy: 0.9254 - val_loss: 0.3263 - val_accuracy: 0.9230

Epoch 00001: val_accuracy improved from 0.92286 to 0.92303, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.33467 to 0.32627, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_loss-weights.hdf5
epoch 18

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 124s 7ms/step - loss: 0.3144 - accuracy: 0.9274 - val_loss: 0.3125 - val_accuracy: 0.9266

Epoch 00001: val_accuracy improved from 0.92303 to 0.92663, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.32627 to 0.31252, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_loss-weights.hdf5
epoch 19

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 126s 7ms/st

17138/17138 [==============================] - 123s 7ms/step - loss: 0.1893 - accuracy: 0.9492 - val_loss: 0.2309 - val_accuracy: 0.9307

Epoch 00001: val_accuracy did not improve from 0.93137

Epoch 00001: val_loss improved from 0.23337 to 0.23085, saving model to D:/mulocdeep/lv1_result11/fold0_big_lv1_loss-weights.hdf5
epoch 38

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 127s 7ms/step - loss: 0.1846 - accuracy: 0.9500 - val_loss: 0.2311 - val_accuracy: 0.9313

Epoch 00001: val_accuracy did not improve from 0.93137

Epoch 00001: val_loss did not improve from 0.23085
epoch 39

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 124s 7ms/step - loss: 0.1806 - accuracy: 0.9516 - val_loss: 0.2311 - val_accuracy: 0.9291

Epoch 00001: val_accuracy did not improve from 0.93137

Epoch 00001: val_loss did not improve from 0.23085
epoch 40

Train on 17138 samples, validate on 24


Epoch 00001: val_accuracy did not improve from 0.93227

Epoch 00001: val_loss did not improve from 0.21969
epoch 62

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 123s 7ms/step - loss: 0.1160 - accuracy: 0.9669 - val_loss: 0.2315 - val_accuracy: 0.9293

Epoch 00001: val_accuracy did not improve from 0.93227

Epoch 00001: val_loss did not improve from 0.21969
epoch 63

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 122s 7ms/step - loss: 0.1131 - accuracy: 0.9681 - val_loss: 0.2285 - val_accuracy: 0.9293

Epoch 00001: val_accuracy did not improve from 0.93227

Epoch 00001: val_loss did not improve from 0.21969
epoch 64

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 121s 7ms/step - loss: 0.1106 - accuracy: 0.9687 - val_loss: 0.2305 - val_accuracy: 0.9259

Epoch 00001: val_accuracy did not improve from 0.93227

E

...

17077/17077 [==============================] - 124s 7ms/step - loss: 0.1134 - accuracy: 0.9674 - val_loss: 0.2376 - val_accuracy: 0.9290

Epoch 00001: val_accuracy did not improve from 0.93089

Epoch 00001: val_loss did not improve from 0.22942
epoch 64

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 124s 7ms/step - loss: 0.1100 - accuracy: 0.9687 - val_loss: 0.2416 - val_accuracy: 0.9281

Epoch 00001: val_accuracy did not improve from 0.93089

Epoch 00001: val_loss did not improve from 0.22942
epoch 65

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 126s 7ms/step - loss: 0.1073 - accuracy: 0.9695 - val_loss: 0.2382 - val_accuracy: 0.9292

Epoch 00001: val_accuracy did not improve from 0.93089

Epoch 00001: val_loss did not improve from 0.22942
epoch 66

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 121s 7ms/st

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 137s 8ms/step - loss: 0.7517 - accuracy: 0.6720 - val_loss: 0.6893 - val_accuracy: 0.7934

Epoch 00001: val_accuracy improved from -inf to 0.79340, saving model to D:/mulocdeep/lv1_result11/fold7_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 0.68933, saving model to D:/mulocdeep/lv1_result11/fold7_big_lv1_loss-weights.hdf5
epoch 1

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 122s 7ms/step - loss: 0.6471 - accuracy: 0.8180 - val_loss: 0.6616 - val_accuracy: 0.8497

Epoch 00001: val_accuracy improved from 0.79340 to 0.84972, saving model to D:/mulocdeep/lv1_result11/fold7_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.68933 to 0.66156, saving model to D:/mulocdeep/lv1_result11/fold7_big_lv1_loss-weights.hdf5
epoch 2

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17

epoch 18

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 114s 7ms/step - loss: 0.3167 - accuracy: 0.9262 - val_loss: 0.3199 - val_accuracy: 0.9247

Epoch 00001: val_accuracy did not improve from 0.92549

Epoch 00001: val_loss improved from 0.33049 to 0.31988, saving model to D:/mulocdeep/lv1_result11/fold7_big_lv1_loss-weights.hdf5
epoch 19

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 121s 7ms/step - loss: 0.3066 - accuracy: 0.9274 - val_loss: 0.3108 - val_accuracy: 0.9243

Epoch 00001: val_accuracy did not improve from 0.92549

Epoch 00001: val_loss improved from 0.31988 to 0.31079, saving model to D:/mulocdeep/lv1_result11/fold7_big_lv1_loss-weights.hdf5
epoch 20

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 117s 7ms/step - loss: 0.2984 - accuracy: 0.9285 - val_loss: 0.2997 - val_accuracy: 0.9264

Epoch 0


Epoch 00001: val_accuracy did not improve from 0.93047

Epoch 00001: val_loss improved from 0.23838 to 0.23832, saving model to D:/mulocdeep/lv1_result11/fold7_big_lv1_loss-weights.hdf5
epoch 39

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 135s 8ms/step - loss: 0.1815 - accuracy: 0.9506 - val_loss: 0.2359 - val_accuracy: 0.9286

Epoch 00001: val_accuracy did not improve from 0.93047

Epoch 00001: val_loss improved from 0.23832 to 0.23590, saving model to D:/mulocdeep/lv1_result11/fold7_big_lv1_loss-weights.hdf5
epoch 40

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 123s 7ms/step - loss: 0.1785 - accuracy: 0.9512 - val_loss: 0.2386 - val_accuracy: 0.9287

Epoch 00001: val_accuracy did not improve from 0.93047

Epoch 00001: val_loss did not improve from 0.23590
epoch 41

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [========================

17053/17053 [==============================] - 122s 7ms/step - loss: 0.1136 - accuracy: 0.9675 - val_loss: 0.2407 - val_accuracy: 0.9303

Epoch 00001: val_accuracy did not improve from 0.93095

Epoch 00001: val_loss did not improve from 0.22477
epoch 64

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 120s 7ms/step - loss: 0.1120 - accuracy: 0.9680 - val_loss: 0.2394 - val_accuracy: 0.9288

Epoch 00001: val_accuracy did not improve from 0.93095

Epoch 00001: val_loss did not improve from 0.22477
epoch 65

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 122s 7ms/step - loss: 0.1084 - accuracy: 0.9696 - val_loss: 0.2382 - val_accuracy: 0.9288

Epoch 00001: val_accuracy did not improve from 0.93095

Epoch 00001: val_loss did not improve from 0.22477
epoch 66

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 119s 7ms/st